### Запрос к API Last.fm

Обратимся к API ведущего музыкального портала Last.fm, который занимается скробблингом (считыванием) композиций из плеера пользователей и ведёт статистику их прослушиваний.

Преследуемые цели:
- получить статистику наиболее прослушиваемых групп и композиций на территории Молдовы и сформировать из неё датафрейм, пригодный для визуализации.
- испытать порядок взаимодействия с API, предоставляемым сайтом Last.fm

In [34]:
#import libraries

import requests
import time
import json
import pandas as pd

Сформируем url-адрес, к которому будет произведён запрос. Параметры запроса укажем в самом адресе.

In [35]:
url = 'http://ws.audioscrobbler.com/2.0/?method=geo.gettopartists&country=moldova&api_key=2689b481c219016f770446b53238ea94&format=json'

Произведём запрос при помощи библиотеки requests. Дождёмся ответа от сервера.

In [36]:
response = requests.get(url = url)
response

<Response [200]>

In [37]:
moldova_data = json.loads(response.content)

Сформируем из получившегося json-ответа датафрейм. Для этого поместим его в метод DataFrame, очистим от лишних данных и изменим начало индексов на единицу.

In [38]:
data_md = pd.DataFrame(moldova_data['topartists']['artist'])
data_md.drop(columns = ['image', 'streamable', 'mbid', 'url'], inplace=True)
data_md.index = [i for i in range(1, len(data) + 1)]
data_md['rate'] = data_md.index
data_md.head(10)

,name,listeners,rate
1,Radiohead,5393887,1
2,Red Hot Chili Peppers,5175779,2
3,Metallica,3315705,3
4,Muse,4528650,4
5,Queen,4807066,5
6,Coldplay,6258348,6
7,Nirvana,4981576,7
8,Linkin Park,4518933,8
9,Lana Del Rey,2698639,9
10,Moby,2857018,10


Сравним данные прослушиваний Молдовы с Российскими и Румынскими прослушиваниями.

In [39]:
url2 = 'http://ws.audioscrobbler.com/2.0/?method=geo.gettopartists&country=romania&api_key=2689b481c219016f770446b53238ea94&format=json'

In [40]:
response2 = requests.get(url = url2)
romania_data = json.loads(response2.content)

{'topartists': {'artist': [{'name': 'Radiohead',
    'listeners': '5393887',
    'mbid': 'a74b1b7f-71a5-4011-9441-d0b5e4122711',
    'url': 'https://www.last.fm/music/Radiohead',
    'streamable': '0',
    'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'small'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'medium'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'large'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'extralarge'},
     {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
      'size': 'mega'}]},
   {'name': 'Pink Floyd',
    'listeners': '3612211',
    'mbid': '83d91898-7763-47d7-b03b-b92132375c47',
    'url': 'https://www.last.fm/music/Pink+Floyd',
    'stream

In [41]:
data_rom = pd.DataFrame(romania_data['topartists']['artist'])
data_rom.drop(columns = ['image', 'streamable', 'mbid', 'url'], inplace=True)
data_rom.index = [i for i in range(1, len(data) + 1)]
data_rom['rate'] = data.index
data_rom.head(10)

,name,listeners,rate
1,Radiohead,5393887,1
2,Pink Floyd,3612211,2
3,Queen,4807066,3
4,Coldplay,6258348,4
5,Red Hot Chili Peppers,5175779,5
6,Metallica,3315705,6
7,David Bowie,3900771,7
8,Arctic Monkeys,4317831,8
9,Massive Attack,2784027,9
10,Lana Del Rey,2698639,10


Сделаем сравнение также и по Российской Федерации. Применим разве что другой подход: пропишем параметры через словарь, а не напрямую в url.

In [42]:
params = {
    'api_key':'2689b481c219016f770446b53238ea94',
    'country':'Russian Federation',
}
url_3 = 'http://ws.audioscrobbler.com/2.0/?method=geo.gettopartists&format=json'
response3 = requests.get(url = url_3, params = params)
russia_data = json.loads(response3.content)
data_rus = pd.DataFrame(russia_data['topartists']['artist'])
data_rus.drop(columns = ['image', 'streamable', 'mbid', 'url'], inplace=True)
data_rus.index = [i for i in range(1, len(data) + 1)]
data_rus['rate'] = data.index
data_rus.head(10)

,name,listeners,rate
1,Linkin Park,4518933,1
2,Red Hot Chili Peppers,5175779,2
3,Queen,4807066,3
4,Radiohead,5393887,4
5,Depeche Mode,3033383,5
6,Nirvana,4981576,6
7,Lana Del Rey,2698639,7
8,Muse,4528650,8
9,Coldplay,6258348,9
10,The Prodigy,2407400,10


#### Сводная таблица по лучшим группам Молдовы, Румынии и России

Посмотрим, как отличаются музыкальные вкусы миллениалов (это только предположение, что last.fm устарел) в Молдове, Румынии и Российской Федерации. 

In [56]:
rating = pd.merge(pd.merge(data_md,data_rom,on='name', how='left'),data_rus,on='name', how='left')
rating.drop(columns=['listeners_x', 'listeners_y', 'listeners'], inplace=True)
rating.rename(columns={'rate_x':'moldova', 'rate_y':'romania', 'rate':'russia'}, inplace=True)
rating

,name,moldova,romania,russia
0,Radiohead,1,1.0,4.0
1,Red Hot Chili Peppers,2,5.0,2.0
2,Metallica,3,6.0,15.0
3,Muse,4,12.0,8.0
4,Queen,5,3.0,3.0
5,Coldplay,6,4.0,9.0
6,Nirvana,7,15.0,6.0
7,Linkin Park,8,24.0,1.0
8,Lana Del Rey,9,10.0,7.0
9,Moby,10,26.0,11.0


Похулиганим и выясним, кто же оказался к Молдове ближе по музыкальным вкусам. Чтобы не углубляться в расчёты, сделаем это на примере пятидесяти первых музыкантов и выведем сумму отсутствующих исполнителей из рейтинга Молдовы в других рейтингах.

In [59]:
rating.isnull().sum()

name        0
moldova     0
romania    11
russia      9
dtype: int64

Из молдавского топ-50 в румынском отсутствуют 11 групп, в российском - 9. В этой импровизированной дуэли Российская Федерация оказалась ближе.